This code calculates and stores cosmological power spectra 
and related coefficients for weak lensing and matter power spectra based on cosmological models.

In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '../LimberCloud/' #'/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

Choosing a cosmological model.

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY_INFO = json.load(file)

COSMOLOGY = pyccl.Cosmology(
    h = COSMOLOGY_INFO['H'],
    w0 = COSMOLOGY_INFO['W0'],
    wa = COSMOLOGY_INFO['WA'], 
    n_s = COSMOLOGY_INFO['NS'], 
    A_s = COSMOLOGY_INFO['AS'],
    m_nu = COSMOLOGY_INFO['M_NU'],  
    Neff = COSMOLOGY_INFO['N_EFF'],
    Omega_b = COSMOLOGY_INFO['OMEGA_B'], 
    Omega_k = COSMOLOGY_INFO['OMEGA_K'], 
    Omega_c = COSMOLOGY_INFO['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

Calculate the power spectrum.

In [ ]:
# Define the redshift grid
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

# Calculate corresponding scale factors and comoving distances
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Calculate the power spectrum on the grid
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=COSMOLOGY, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

In [5]:
# Numerical integral

def integral_I3(chi1, chi2, chi3, chi4, chi5, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        result = (chi2 ** 2 - chi ** 2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1)
        
        result = result * ((chi5 - chi3) / 2 - chi * chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4) + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3))
        
        result = result * (((chi / chi2) ** 3) * power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    def formula_n(chi):
        result = (chi2 ** 2 - chi ** 2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1)
        
        result = result * ((chi5 - chi3) / 2 - chi * chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4) + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

In [6]:
# Coefficient

def coefficient_J3(chi1, chi2, chi3, chi4, chi5, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = (chi5 - chi3) / (2 * chi2)
    c = chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3) - chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4)
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (5 * c * (112 + z * ( - 104 + 27 * z)) + 4 * b * (294 + z * ( - 308 + 89 * z))) / 705600
    else:
        formula = - ((1 / (2 * a ** 4)) * ((1 / 12600) * (a * (35 * a ** 2 * (10 * a * (6 * (6 + a * ( - 9 + 2 * a)) * b + (24 + a * ( - 60 + (44 - 9 * a) * a)) * c) - 
(10 * (12 + a * (6 + a * ( - 32 + 9 * a))) * b + (60 + a * (30 + a * ( - 340 + 9 * (35 - 8 * a) * a))) * c) * p) - 
14 * a * (50 * a * (12 + a * (6 + a * ( - 32 + 9 * a))) * b - 5 * a * ( - 60 + a * ( - 30 + a * (340 + 9 * a * ( - 35 + 8 * a)))) * c - 
(5 * (60 + a * (30 + a * (20 + 9 * a * ( - 25 + 8 * a)))) * b + 2 * (60 + a * (30 + a * (20 - 3 * a * (195 - 204 * a + 50 * a ** 2)))) * c) * p) * z + 
(7 * a * (5 * (60 + a * (30 + a * (20 + 9 * a * ( - 25 + 8 * a)))) * b + 2 * (60 + a * (30 + a * (20 - 3 * a * (195 - 204 * a + 50 * a ** 2)))) * c) - 
(21 * (60 + a * (30 + a * (20 + a * (15 + 4 * a * ( - 72 + 25 * a))))) * b + (420 + a * (210 + a * (140 + a * (105 - 8 * a * (777 + 25 * a * ( - 35 + 9 * a)))))) * c) * p) * z ** 2)) + (1 / 30) * ( - 1 + a) ** 2 * (5 * a ** 2 * (6 * a * b - 4 * ( - 1 + a) * a * c - ((2 + 4 * a) * b + c + (2 - 3 * a) * a * c) * p) + 
2 * a * ( - 10 * a * (1 + 2 * a) * b + 5 * ( - 1 + a) * a * (1 + 3 * a) * c + 5 * (1 + a * (2 + 3 * a)) * b * p - 2 * ( - 1 + a) * (1 + 3 * a + 6 * a ** 2) * c * p) * z + 
(5 * a * (1 + a * (2 + 3 * a)) * b - 2 * ( - 1 + a) * a * (1 + 3 * a + 6 * a ** 2) * c - (3 * (1 + a * (2 + a * (3 + 4 * a))) * b + c + a * (2 + a * (3 + 2 * (2 - 5 * a) * a)) * c) * p) * z ** 2) * numpy.log(1 - a)))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])
CHI3 = numpy.float128(CHI_GRID[GRID_INDEX + 2])
CHI4 = numpy.float128(CHI_GRID[GRID_INDEX + 3])
CHI5 = numpy.float128(CHI_GRID[GRID_INDEX + 4])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 4957.1305889213954288 4973.4637436395141776 4989.717316369170476 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I3(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J3(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

489.97128143090777713 489.97124404262571432 -7.630708876165022414e-08


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_INDEX + 3]
CHI5 = CHI_GRID[GRID_INDEX + 4]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 88.58943766284152 132.5655204109753 176.3277722724983 0.0 1826.8838600584127 0.0 0.01


In [ ]:
# Case2: chi1 = 0

INTEGRAL = integral_I3(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J3(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

222556.920354277 222556.92035429936 1.0036416142611415e-13
